# PCA & MFCC

In [ ]:
pathAudioHalf = 'D:/AudioIRA/dataset7.new.0.5/'

pathAudioOne = 'D:/AudioIRA/dataset7.new.1/'

pathAudioThree = 'D:/AudioIRA/dataset7.new.3/'

pathAudioFive = 'D:/AudioIRA/dataset7.new..5/'


# Basic Libraries

In [ ]:
from sklearn.decomposition import PCA

from sklearn.neural_network import MLPClassifier
import pandas as pd

from sklearn.model_selection import KFold
import IPython.display as ipd
from pywt import wavedec
import pywt
import copy
from scipy.io import wavfile

import random
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from sklearn.metrics import confusion_matrix
from pydub import AudioSegment
import wave
import librosa
import numpy as np
import sklearn
import librosa.display
import matplotlib.pyplot as plt
import sklearn
import warnings
from glob import glob
from librosa import feature
import csv
import pandas as pd
import os
warnings.filterwarnings('ignore')


pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

# Libraries for Classification and building Models

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Functions For Audio

In [ ]:
class AudioUtil():
    # ----------------------------
    # Load an audio file. Return the signal as a tensor and the sample rate
    # ----------------------------
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)

    @staticmethod
    def resample(aud, newsr):
        sig, sr = aud

        if (sr == newsr):
          # Nothing to do
          return aud

        num_channels = sig.shape[0]
        # Resample first channel
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1, :])
        if (num_channels > 1):
            # Resample the second channel and merge both channels
            retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:, :])
            resig = torch.cat([resig, retwo])

        return ((resig, newsr))

    @staticmethod
    def rechannel(aud, new_channel):
        sig, sr = aud

        if (sig.shape[0] == new_channel):
          # Nothing to do
          return aud

        if (new_channel == 1):
          # Convert from stereo to mono by selecting only the first channel
          resig = sig[:1, :]
        else:
          # Convert from mono to stereo by duplicating the first channel
          resig = torch.cat([sig, sig])

        return ((resig, sr))


    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec

    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig, sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)
    @staticmethod
    def apply_LowPassFilter(aud):
        transform = Compose(
            [LowPassFilter(sample_rate=aud[1])],
        )
        t_audio = transform(aud[0])
        # torchaudio.save("tests/filter.wav", t_audio, sample_rate=sample_rate)
        assert t_audio.shape == aud[0].shape

        return t_audio , aud[1]
    @staticmethod
    def lowpassfilter(signal, thresh = 0.63, wavelet="db2",level=4):
        thresh = thresh*np.nanmax(signal)
        coeff = pywt.wavedec(signal, wavelet, mode="per" ,level=level)
        coeff[1:] = (pywt.threshold(i, value=thresh, mode="soft" ) for i in coeff[1:])
        reconstructed_signal = pywt.waverec(coeff, wavelet, mode="per")
        return reconstructed_signal




# Basic Function

In [ ]:
def readDataFirst(num,level=2,thresh=1,augmentData=False , dwt = False , dwtMfcc = False , mfcc = False):
    if(num == 0.5):
        data_dir = pathAudioHalf
        folders = glob(data_dir+'/*')

        feat = []
        lab = []
    if(num == 1):
        data_dir = pathAudioOne
        folders = glob(data_dir+'/*')

        feat = []
        lab = []
    if(num == 3):
        data_dir = pathAudioThree
        folders = glob(data_dir+'/*')

        feat = []
        lab = []
    if(num == 5):
        data_dir = pathAudioFive
        folders = glob(data_dir+'/*')

        feat = []
        lab = []
        
        
    i=-1
    for folder in folders:
        audio_files=glob(folder + '/*.wav')
        i=i+1
        for file in audio_files:
            
            
            if dwtMfcc:
                sr,signal=wavfile.read(file)
                if len(signal.shape) == 2:
                    signal = signal.T
                    signal = signal[0]
                aud2 = AudioUtil.lowpassfilter(signal,thresh = thresh,level = level)        
                aud2 = aud2/max(aud2)
                aud2=aud2.reshape(1,aud2.shape[0])
                tens=torch.tensor(aud2,dtype=torch.float32)
                aud=(tens,sr)

                reaud = AudioUtil.resample(aud, 16000)
                rechan = AudioUtil.rechannel(reaud, 2)
                sgram = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None).T
                s = np.mean(np.array(sgram),axis=0)
                s.reshape(128)
                feat.append(s)
                lab.append(i)
                
                
                
            if dwt:
                sr,signal=wavfile.read(file)
#                 print(file)
                if len(signal.shape) == 2:
                    signal = signal.T
                    signal = signal[0]
                aud2 = AudioUtil.lowpassfilter(signal,thresh = 0.01,level = 2)        
                aud2 = aud2/max(aud2)
                aud2=aud2.reshape(1,aud2.shape[0])
                tens=torch.tensor(aud2,dtype=torch.float32)
                aud=(tens,sr)

                reaud = AudioUtil.resample(aud, 16000)
                rechan = AudioUtil.rechannel(reaud, 2)
                
                if(len(rechan[0][0]) % 128 !=0):
                    ss = len(rechan[0][0]) % 128
                    j= rechan[0][0].resize_((len(rechan[0][0])+(128-ss)))
                    k=rechan[0][1].resize_((len(rechan[0][1])+(128-ss)))

                    j = np.asarray(j)
                    res = len(j)/128

                    re = j.reshape([int(res),64,2])
                    re = np.mean(np.array(re),axis=0)
                    re = re.reshape(128)
                    s = re





                else:
#                     res = rechan[0].shape[1]/128
#                     re = rechan[0].reshape([int(res),64,2])
                    
                    j= rechan[0][0]
                    j = np.asarray(j)
                    res = len(j)/128
                    
                    re = j.reshape([int(res),64,2])
                    re = np.mean(np.array(re),axis=0)
                    re = re.reshape(128)
                    s = re
                    
                    
                feat.append(s)
                lab.append(i)
                
            
            if mfcc : 
                aud = AudioUtil.open(file)
                reaud = AudioUtil.resample(aud, 16000)
                rechan = AudioUtil.rechannel(reaud, 2)
                sgram = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None).T
                s = np.mean(np.array(sgram),axis=0)
                s = s.reshape(128)
                feat.append(s)
                lab.append(i)
            

            
            
            if augmentData:
                sgram2 = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None)
                aug_sgram = AudioUtil.spectro_augment(sgram2, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2).T
                s2 = np.mean(np.array(aug_sgram),axis=0)
                s2 = s2.reshape(128)
                feat.append(s2)
                lab.append(i)
            
                
            
    
    return [feat , lab]




# Model

In [ ]:
def intiModel(X_train , X_test , Y_train , Y_test , PCA = False):
    
    if PCA :         
        X_train = X_train.reshape(X_train.shape[0], 4, 8, 1)
        X_test = X_test.reshape(X_test.shape[0], 4, 8, 1)

        input_dim = (4, 8, 1)
    else : 
        X_train = X_train.reshape(X_train.shape[0], 16, 8, 1)
        X_test = X_test.reshape(X_test.shape[0], 16, 8, 1)

        input_dim = (16, 8, 1)

    model = Sequential()

    model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(256, (3, 3), padding = "same", activation = "tanh"))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(1024, activation = "tanh"))
    model.add(Dense(Y_train.shape[1], activation = "softmax"))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model , X_train , X_test ,Y_train , Y_test

In [ ]:
def cumulative_explained_variance(X):
    pca = PCA().fit(X)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance');

In [ ]:
def get_pca_projection(n_compnent,X):
    pca = PCA(n_compnent)  # project from 64 to n_compnent dimensions
    projected = pca.fit_transform(X)
    print ('original data shape : ')
    print(X.shape)
    print ('projected data shape : ')
    print(projected.shape)
    return projected

# Draw Functions

## draw History

In [ ]:
def drawHistory(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

## draw all historys

### model accuracy

In [ ]:
def drawALLHistoryAccuracy(history):
    for i in range(5):
        plt.plot(history[i].history['accuracy'])
    plt.title('train model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['KFold1', 'KFold2','KFold3','KFold4','KFold5'], loc='upper left')
    plt.show()
    
    for i in range(5):
        plt.plot(history[i].history['val_accuracy'])
    plt.title('test model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['KFold1', 'KFold2','KFold3','KFold4','KFold5'], loc='upper left')
    plt.show()

### model loss

In [ ]:
def drawALLHistoryLoss(history):
    for i in range(5):
        plt.plot(history[i].history['loss'])
    plt.title('train model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['KFold1', 'KFold2','KFold3','KFold4','KFold5'], loc='upper left')
    plt.show()
    
    for i in range(5):
        plt.plot(history[i].history['val_loss'])
    plt.title('test model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['KFold1', 'KFold2','KFold3','KFold4','KFold5'], loc='upper left')
    plt.show()

## draw Confusion Matrix

In [ ]:
def drawConfusionMatrix(Y_test,preds):
    confusion_mc = confusion_matrix(Y_test, preds)
    # put it in data frame to visualize
    df_cm = pd.DataFrame(confusion_mc, 
                         index = [i for i in range(0,confusion_mc.shape[0])], columns = [i for i in range(0,confusion_mc.shape[0])])
    plt.figure(figsize=(20,20))
    sns.heatmap(df_cm, annot=True,)
    plt.title('SVM Linear Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(Y_test,preds)))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Initializing the variables model

In [ ]:
def intialVariabels(num, augmentData=False ,dwt = False,mfcc=False,dwtMfcc=False):    
    
    #read and init data to half second model
    temp = readDataFirst(num = num , level=1 , thresh=0.01 , augmentData = augmentData ,dwt = dwt,dwtMfcc=dwtMfcc,mfcc=mfcc)
    temp = np.array(temp)
    data = temp.transpose()
    X_ = data[:, 0]
    Y = data[:, 1]

    X = np.empty([X_.shape[0], 128])
    for i in range(X_.shape[0]):
        X_[i] = np.reshape(X_[i] , (128))
        X[i] = (X_[i]) 

    Y = to_categorical(Y)



    print('All Data leangth is : {}'.format(X_.shape[0]))
    print()
    print('categorical leangth is : {}'.format(Y.shape[0]))
    
    return X , Y 




# Kfold Fiting Model Cnn

In [ ]:
def fitingModelCnn(X , Y , PCA = False):
    #scores & history & confusionMatrix & BPNN data X,Y & historyBPNN
    scores = []
    history = []
    confusionMatrix = []
    BPNN = []
    
    
    cv = KFold(n_splits=5, random_state=42, shuffle=True)
    iKFold = 1 
    for train_index, test_index in cv.split(X):
        #split data to train & test  from KFOLD Split
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
        print('Train Data leangth for KFold :{} : is {}'.format(iKFold , X_train.shape[0]))
        print('Test Data leangth for KFold :{} : is {}'.format(iKFold , X_test.shape[0]))
        print()
        print()

        #intializing cnn model to half second model
        halfSecondModel, halfSecondX_train , halfSecondX_test ,halfSecondY_train ,halfSecondY_test = intiModel(X_train , X_test , y_train , y_test , PCA = PCA)

        #fit model
        historyT = halfSecondModel.fit(halfSecondX_train, halfSecondY_train, epochs = 7, batch_size = 50, validation_data = (halfSecondX_test, halfSecondY_test))

        #evaluate model
        ModelScore = halfSecondModel.evaluate(halfSecondX_test, halfSecondY_test)

        #save result
        scores.append(ModelScore[1])
        history.append(historyT)

        #save for confusionMatrix
        predictions = halfSecondModel.predict(halfSecondX_test)
        predictions = np.argmax(predictions, axis = 1)
        Y_test_max = np.argmax(halfSecondY_test, axis = 1)
        confusionMatrix.append([Y_test_max,predictions])    

        #save for BPNN
        BPNN.append([halfSecondX_train, halfSecondX_test, halfSecondY_train, halfSecondY_test])


        iKFold = iKFold +1
        
    return scores, history , confusionMatrix , BPNN



# DWT Functions

In [ ]:
def showSignal(file):
    sr,signal=wavfile.read(file);
    (cA1, cD1) = pywt.dwt(signal, 'db2', 'smooth')
    reconstructed_signal = pywt.idwt(cA1, cD1, 'db2', 'smooth')
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(signal, label='signal')
    ax.plot(reconstructed_signal, label='reconstructed signal', linestyle='--')
    ax.legend(loc='upper left')
    plt.show()
    
    return signal , sr

In [ ]:
def showMultiLevel(chirp_signal):
    fig, ax = plt.subplots(figsize=(6,1))
    ax.set_title("Original Chirp Signal: ")
    ax.plot(chirp_signal)
    plt.show()

    data = chirp_signal
    waveletname = 'sym5'

    fig, axarr = plt.subplots(nrows=5, ncols=2, figsize=(6,6))
    for ii in range(5):
        (data, coeff_d) = pywt.dwt(data, waveletname)
        axarr[ii, 0].plot(data, 'r')
        axarr[ii, 1].plot(coeff_d, 'g')
        axarr[ii, 0].set_ylabel("Level {}".format(ii + 1), fontsize=14, rotation=90)
        axarr[ii, 0].set_yticklabels([])
        if ii == 0:
            axarr[ii, 0].set_title("Approximation coefficients", fontsize=14)
            axarr[ii, 1].set_title("Detail coefficients", fontsize=14)
        axarr[ii, 1].set_yticklabels([])
    plt.tight_layout()
    plt.show()

In [ ]:
def showDWT(signal):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(signal, color="b", alpha=0.5, label='original signal')
    rec = AudioUtil.lowpassfilter(signal, thresh=0.01,level=2)
    ax.plot(rec, 'k', label='DWT smoothing}', linewidth=1)
    ax.legend()
    ax.set_title('Removing High Frequency Noise with DWT', fontsize=18)
    ax.set_ylabel('Signal Amplitude', fontsize=16)
    ax.set_xlabel('Sample No', fontsize=16)
    plt.show()
    ipd.Audio(rec, rate=sr)

# save accuracy

In [ ]:
def saveAccuracy(scores , all_history , file , isCNN ):
    iKFold = 0
    score = 0
    indexMaxScore = 0
    for sc in scores:
        if sc > score:
            score = sc
            indexMaxScore = iKFold
        iKFold = iKFold +1
    bestScore_halfSecond = all_history[indexMaxScore]
    

    # convert the history.history dict to a pandas DataFrame:
    if isCNN :
        hist_df = pd.DataFrame(bestScore_halfSecond.history)
    else : 
        hist_df = pd.DataFrame(bestScore_halfSecond.validation_scores_) 
    #save to csv: 
    #hist_csv_file = file
    #with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(file,index=False)

# show all accuracy levels

## CNN accuracy levels

In [ ]:
def ALLCNNAccuracyLevels(histLevel1 , histLevel2 ,title1 , title2 ,nLevel1 , nLevel2):
    plt.plot(histLevel1['accuracy'])      
    plt.plot(histLevel2['accuracy'])

    plt.title(title1)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend([nLevel1 , nLevel2 ], loc='upper left')
    plt.show()
    
    plt.plot(histLevel1['val_accuracy'])      
    plt.plot(histLevel2['val_accuracy'])
    
    plt.title(title2)
    plt.ylabel('val_accuracy')
    plt.xlabel('epoch')
    plt.legend([nLevel1 , nLevel2], loc='upper left')
    plt.show()

# 1- DWT

In [ ]:
# if we need augment just change augmentData=True

X_dwt , Y_dwt = intialVariabels(3 , augmentData=False ,dwt = True)

In [ ]:
scores_dwt, history_dwt , confusionMatrix_dwt , BPNN_dwt = fitingModelCnn(X_dwt , Y_dwt)

### evaluate model

In [ ]:
iKFold =1
for ev in scores_dwt:
    print("this is score for :{} KFold:".format(iKFold))
    print(ev)
    print()
    iKFold+=1

### draw all history one by one

In [ ]:
iKFold =1
for hist in history_dwt:
    print("this is history for :{} KFold:".format(iKFold))
    drawHistory(hist)
    print()
    iKFold =iKFold + 1

### draw all history accuracy

In [ ]:
drawALLHistoryAccuracy(history_dwt)

### draw all history loss

In [ ]:
drawALLHistoryLoss(history_dwt)

### draw confusion matrix 

In [ ]:
iKFold =1
for conv in confusionMatrix_dwt:
    print("this is confusion Matrix for :{} KFold:".format(iKFold))
    drawConfusionMatrix(conv[0] , conv[1])
    print()
    iKFold+=1

# 2- DWT & PCA

In [ ]:
cumulative_explained_variance(X_dwt)

In [ ]:
X_projected_dwt=get_pca_projection(32,X_dwt)

In [ ]:
scores_dwtPca, history_dwtPca , confusionMatrix_dwtPca , BPNN_dwtPca = fitingModelCnn(X_projected_dwt , Y_dwt , PCA = True)

### evaluate model

In [ ]:
iKFold =1
for ev in scores_dwtPca:
    print("this is score for :{} KFold:".format(iKFold))
    print(ev)
    print()
    iKFold+=1

### draw all history one by one

In [ ]:
iKFold =1
for hist in history_dwtPca:
    print("this is history for :{} KFold:".format(iKFold))
    drawHistory(hist)
    print()
    iKFold =iKFold + 1

### draw all history accuracy

In [ ]:
drawALLHistoryAccuracy(history_dwtPca)

### draw all history accuracy

### draw all history loss

In [ ]:
drawALLHistoryLoss(history_dwtPca)

### draw confusion matrix 

In [ ]:
iKFold =1
for conv in confusionMatrix_dwtPca:
    print("this is confusion Matrix for :{} KFold:".format(iKFold))
    drawConfusionMatrix(conv[0] , conv[1])
    print()
    iKFold+=1

# 3- DWT & MFCC

In [ ]:
# if we need augment just change augmentData=True

X_DwtMfcc , Y_DwtMfcc = intialVariabels(3 , augmentData=False ,dwtMfcc = True)

In [ ]:
scores_DwtMfcc, history_DwtMfcc , confusionMatrix_DwtMfcc , BPNN_DwtMfcc = fitingModelCnn(X_DwtMfcc , Y_DwtMfcc)

### evaluate model

In [ ]:
iKFold =1
for ev in scores_DwtMfcc:
    print("this is score for :{} KFold:".format(iKFold))
    print(ev)
    print()
    iKFold+=1

### draw all history one by one

In [ ]:
iKFold =1
for hist in history_DwtMfcc:
    print("this is history for :{} KFold:".format(iKFold))
    drawHistory(hist)
    print()
    iKFold =iKFold + 1

### draw all history accuracy

In [ ]:
drawALLHistoryAccuracy(history_DwtMfcc)

### draw all history accuracy

### draw all history loss

In [ ]:
drawALLHistoryLoss(history_DwtMfcc)

### draw confusion matrix 

In [ ]:
iKFold =1
for conv in confusionMatrix_DwtMfcc:
    print("this is confusion Matrix for :{} KFold:".format(iKFold))
    drawConfusionMatrix(conv[0] , conv[1])
    print()
    iKFold+=1

# 4- MFCC & PCA

In [ ]:
# if we need augment just change augmentData=True

X_Mfcc , Y_Mfcc = intialVariabels(3 , augmentData=False ,mfcc = True)

In [ ]:
cumulative_explained_variance(X_Mfcc)

In [ ]:
X_projected_mfccPca=get_pca_projection(32,X_Mfcc)

In [ ]:
scores_mfccPca, history_mfccPca , confusionMatrix_mfccPca , BPNN_mfccPca = fitingModelCnn(X_projected_mfccPca , Y_Mfcc)

### evaluate model

In [ ]:
iKFold =1
for ev in scores_mfccPca:
    print("this is score for :{} KFold:".format(iKFold))
    print(ev)
    print()
    iKFold+=1

### draw all history one by one

In [ ]:
iKFold =1
for hist in history_mfccPca:
    print("this is history for :{} KFold:".format(iKFold))
    drawHistory(hist)
    print()
    iKFold =iKFold + 1

### draw all history accuracy

In [ ]:
drawALLHistoryAccuracy(history_mfccPca)

### draw all history accuracy

### draw all history loss

In [ ]:
drawALLHistoryLoss(history_mfccPca)

### draw confusion matrix 

In [ ]:
iKFold =1
for conv in confusionMatrix_mfccPca:
    print("this is confusion Matrix for :{} KFold:".format(iKFold))
    drawConfusionMatrix(conv[0] , conv[1])
    print()
    iKFold+=1

# Save Accuracy

### save best DWT accuracy

In [ ]:
saveAccuracy(scores_Dwt , history_Dwt , 'three_level2_DWT.csv' , True)

### save best DWT& PCA accuracy

In [ ]:
saveAccuracy(scores_dwtPca , history_dwtPca , 'three_level2_PCA&DWT.csv' , True)

### save best DWT& MFCC accuracy

In [ ]:
saveAccuracy(scores_DwtMfcc , history_DwtMfcc , 'three_level2_DWT&MFCC.csv' , True)

### save best MFCC& PCA accuracy

In [ ]:
saveAccuracy(scores_mfccPca , history_mfccPca , 'three_level2_PCA&MFCC.csv' , True)